In [1]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.39  Python-3.11.5 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Setup complete  (16 CPUs, 31.8 GB RAM, 989.6/1862.2 GB disk)


In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

best_model = YOLO("best_seg.pt")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

width = 384
height = 512
margin = 10

new_width = width + margin * 2
new_height = height + margin * 2

in_dirs = ["psa10_images", "psa9_images", "psa8_images", "psa7_images"]
out_dir = "segmented_images"

# Ensure the main output directory exists
os.makedirs(out_dir, exist_ok=True)

for in_dir in in_dirs:
    # Create output subdirectory for this PSA grade
    out_subdir = os.path.join(out_dir, in_dir)
    os.makedirs(out_subdir, exist_ok=True)
    
    # Run inference on the input directory
    results = best_model(in_dir)
    
    len_results = len(results)
    success_count = 0
    fail_count = 0

    for i, result in enumerate(results):
        print(f"Processing Image {i+1}/{len_results} in {in_dir}")
        boxes = result.boxes
    
        # Check if exactly one card detected of class 'Front' (class 0)
        if len(boxes) == 1 and boxes.cls[0] == 0 and boxes.conf[0] >= 0.97:
            polygon_points = result.masks[0].xy[0]  # first detected mask polygon
            polygon = polygon_points.reshape(-1, 1, 2).astype(np.float32)
    
            # Approximate to quadrilateral
            epsilon = 0.02 * cv2.arcLength(polygon, True)
            approx = cv2.approxPolyDP(polygon, epsilon, True)
    
            if len(approx) == 4:
                def order_points(pts):
                    rect = np.zeros((4, 2), dtype="float32")
                    s = pts.sum(axis=1)
                    diff = np.diff(pts, axis=1)
    
                    rect[0] = pts[np.argmin(s)]   # top-left
                    rect[2] = pts[np.argmax(s)]   # bottom-right
                    rect[1] = pts[np.argmin(diff)]# top-right
                    rect[3] = pts[np.argmax(diff)]# bottom-left
                    return rect
    
                approx_points = approx.reshape(-1, 2)
                ordered_pts = order_points(approx_points)
    
                # Define the destination points with margins
                dst = np.array([
                    [margin, margin],
                    [new_width - 1 - margin, margin],
                    [new_width - 1 - margin, new_height - 1 - margin],
                    [margin, new_height - 1 - margin]
                ], dtype="float32")
    
                # Compute perspective transform
                M = cv2.getPerspectiveTransform(ordered_pts, dst)
    
                original_image = result.orig_img
    
                # Warp the image to get a flattened card with margins
                flattened_card = cv2.warpPerspective(original_image, M, (new_width, new_height))
    
                # Save the resulting image instead of showing it
                out_filename = f"processed_{i}.png"
                out_path = os.path.join(out_subdir, out_filename)
                cv2.imwrite(out_path, flattened_card)
                
                success_count += 1
            else:
                print("Could not approximate the detected mask to a quadrilateral.")
                fail_count += 1
        else:
            print("No suitable card detected with the desired confidence and class.")
            fail_count += 1

    # After processing all images in this PSA directory
    print(f"Results for {in_dir}:")
    print(f"  Successfully processed images: {success_count}")
    print(f"  Failed images: {fail_count}")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1152 C:\Users\Brendan\Documents\GitHub\ebay-scraper-demo\psa10_images\$_3.JPG: 640x480 1 front, 127.0ms
image 2/1152 C:\Users\Brendan\Documents\GitHub\ebay-scraper-demo\psa10_images\$_3_1.JPG: 640x480 1 front, 23.6ms
image 3/1152 C:\Users\Brendan\Documents\GitHub\ebay-scraper-demo\psa10_images\$_3_10.JPG: 640x480 1 front, 25.5ms
image 4/1152 C:\Users\Brendan\Documents\GitHub\ebay-scraper-demo\psa10_images\$_3_100.JPG: 640x384 1 front, 118.1ms
ima